In [ ]:
import tushare as ts
import plotly
import plotly.graph_objs as go
from collections import namedtuple

scatter_info = namedtuple('scatter_info', ['x_pos', 'y_pos', 'name', 'color', 'opacity'])

ts.set_token('beb6b9371b4c0dd611ed4985441301f29466e0f69bf7ad5b24be7ca3')
pro = ts.pro_api()
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
def date_format_transfer(date, split_simbol='-'):
    return ["{}{}{}{}{}".format(a[:4], split_simbol, a[4:6], split_simbol, a[6:8]) for a in date]

In [ ]:
def get_day_of_date(date):
    return [a[6:8] for a in date]

In [ ]:
def get_month_daily_datum(pro, ts_code, year, month, fields):
    start_date = "{}{}01".format(year, month)
    end_date = "{}{}31".format(year, month)
    return pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date, fields=fields)

In [ ]:
def draw_diagram(date_infos):
    datum = [go.Scatter(
                x = info.x_pos,
                y = info.y_pos,
                name = info.name,
                line = dict(color = info.color),
                opacity = info.opacity) for info in date_infos]
    plotly.offline.iplot(datum)

In [ ]:
fields = "trade_date, pct_chg"
date9 = get_month_daily_datum(pro, '002236.SZ', '2018', '09', fields)
date9_info = scatter_info(get_day_of_date(date9.trade_date), date9.pct_chg, 'Sep', '#FF0000', 0.8)
date10 = get_month_daily_datum(pro, '002236.SZ', '2018', '10', fields)
date10_info = scatter_info(get_day_of_date(date10.trade_date), date10.pct_chg, 'Oct', '#00FF00', 0.8)
date11 = get_month_daily_datum(pro, '002236.SZ', '2018', '11', fields)
date11_info = scatter_info(get_day_of_date(date11.trade_date), date11.pct_chg, 'Nov', '#0000FF', 0.8)
date_infos = [date9_info, date10_info, date11_info] 
draw_diagram(date_infos)

In [ ]:
df = ts.pro_bar(pro_api=pro, ts_code='002236.SZ', start_date='20181120', end_date='20181128')
dfq = ts.pro_bar(pro_api=pro,ts_code='002236.SZ', adj='qfq', start_date='20181120', end_date='20181128')
dfh = ts.pro_bar(pro_api=pro,ts_code='002236.SZ', adj='hfq', start_date='20181120', end_date='20181128')
print(df)
print(dfq)
print(dfh)


In [ ]:
print(pro.pledge_stat(ts_code='002236.SZ'))


In [ ]:
fields = "trade_date, pct_chg"

data1 = pro.daily(ts_code='002236.SZ', start_date='20180101', end_date='20181131', fields=fields)

pct_chg_st = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for a in data1.pct_chg:
    if a < -9:
        pct_chg_st[0]+=1
    elif a < -8:
        pct_chg_st[1]+=1
    elif a < -7:
        pct_chg_st[2]+=1
    elif a < -6:
        pct_chg_st[3]+=1
    elif a < -5:
        pct_chg_st[4]+=1
    elif a < -4:
        pct_chg_st[5]+=1
    elif a < -3:
        pct_chg_st[6]+=1
    elif a < -2:
        pct_chg_st[7]+=1
    elif a < -1:
        pct_chg_st[8]+=1
    elif a < 0:
        pct_chg_st[9]+=1
    elif a < 1:
        pct_chg_st[10]+=1
    elif a < 2:
        pct_chg_st[11]+=1
    elif a < 3:
        pct_chg_st[12]+=1
    elif a < 4:
        pct_chg_st[13]+=1
    elif a < 5:
        pct_chg_st[14]+=1
    elif a < 6:
        pct_chg_st[15]+=1
    elif a < 7:
        pct_chg_st[16]+=1
    elif a < 8:
        pct_chg_st[17]+=1
    elif a < 9:
        pct_chg_st[18]+=1
    else:
        pct_chg_st[19]+=1

pct_chg_avg_plus = 0
pct_chg_avg_minus = 0
for b in data1.pct_chg:
    if b > 0:
        pct_chg_avg_plus+=b
    else:
        pct_chg_avg_minus+=b

pct_chg_avg_plus = pct_chg_avg_plus / (pct_chg_st[10] + pct_chg_st[11] + pct_chg_st[12] + pct_chg_st[13]
                                      + pct_chg_st[14] + pct_chg_st[15] + pct_chg_st[16] + pct_chg_st[17]
                                      + pct_chg_st[18] + pct_chg_st[19])

pct_chg_avg_minus = pct_chg_avg_minus / (pct_chg_st[0] + pct_chg_st[1] + pct_chg_st[2] + pct_chg_st[3]
                                      + pct_chg_st[4] + pct_chg_st[5] + pct_chg_st[6] + pct_chg_st[7]
                                      + pct_chg_st[8] + pct_chg_st[9])

data1_line = go.Scatter(
                x = [a for a in range(-9,11)],
                y = pct_chg_st,
                name = "overyear",
                line = dict(color = '#00FF00'),
                opacity = 0.8)

data = [data1_line]
plotly.offline.iplot(data)
print("pct chg plus avg is {}, pct chg minus avg is {}".format(pct_chg_avg_plus, pct_chg_avg_minus))
print(sum(pct_chg_st))